In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_miocardial()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.60)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (1700, 110)
5.62% missing data
Class distribution: (array([0, 1]), array([1429,  271]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 2),
                       (46, 9, 1),
                       (96, 9, 5),
                       (96, 11, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 6.00 sec - Loss 0.132941 - ACC 67.86% - ACC Mean 67.86% - AUC 71.94% - AUC Mean 71.94% - Deter 000
Ite 00050 - 0.91 sec - Loss 0.044549 - ACC 67.11% - ACC Mean 67.78% - AUC 70.30% - AUC Mean 71.60% - Deter 036
Ite 00064 - 1.18 sec - Loss 0.036447 - ACC 66.04% - ACC Mean 67.24% - AUC 71.27% - AUC Mean 70.87% - Deter 050
Early stop ite 64, rollback to correction of ite 14, whith acc of 70.5% and auc of 75.03%
OUR METHOD RUN 1/10 - acc: 70.5% +- 0.0% - auc: 75.03% +- 0.0%
Ite 00000 - 1.09 sec - Loss 0.151011 - ACC 67.38% - ACC Mean 67.38% - AUC 69.99% - AUC Mean 69.99% - Deter 000
Ite 00050 - 0.95 sec - Loss 0.065088 - ACC 68.87% - ACC Mean 67.78% - AUC 69.99% - AUC Mean 71.95% - Deter 016
Ite 00084 - 1.11 sec - Loss 0.040206 - ACC 68.49% - ACC Mean 67.31% - AUC 73.39% - AUC Mean 70.86% - Deter 050
Early stop ite 84, rollback to correction of ite 34, whith acc of 69.59% and auc of 74.89%
OUR METHOD RUN 2/10 - acc: 69.59% +- 0.0% - auc: 74.89% +- 0.0%
Ite 00000 - 1.01 sec - Los

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 72.64% +- 0.0% - auc: 75.22% +- 0.0%
MEAN - acc: 72.03% +- 0.0% - auc: 76.72% +- 0.0%
KNN - acc: 64.1% +- 0.0% - auc: 68.1% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:59<00:00, 84.71it/s]



GAIN RUN 1/10 - acc: 64.47% +- 2.85% - auc: 67.24% +- 4.84%



100%|██████████| 5000/5000 [01:00<00:00, 83.04it/s]



GAIN RUN 2/10 - acc: 58.5% +- 2.96% - auc: 62.83% +- 2.81%



100%|██████████| 5000/5000 [00:59<00:00, 83.67it/s] 


...


GAIN RUN 8/10 - acc: 62.97% +- 2.79% - auc: 67.37% +- 3.63%



100%|██████████| 5000/5000 [01:00<00:00, 83.27it/s]



GAIN RUN 9/10 - acc: 59.82% +- 4.74% - auc: 64.79% +- 4.23%



100%|██████████| 5000/5000 [01:05<00:00, 76.32it/s]



GAIN RUN 10/10 - acc: 60.39% +- 1.91% - auc: 62.22% +- 1.74%

GAIN GLOBAL - acc: 61.34% +- 1.84% - auc: 65.19% +- 1.85%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 71.9% +- 0.0% - auc: 76.58% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 1/10 - acc: 69.53% +- 2.68% - auc: 72.79% +- 3.4%

Iteration: 0
Iteration: 1
Iteration: 2

MISSFOREST RUN 2/10 - acc: 68.67% +- 2.3% - auc: 71.46% +- 3.8%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 3/10 - acc: 67.06% +- 2.53% - auc: 71.55% +- 2.48%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 4/10 - acc: 70.43% +- 2.26% - auc: 75.27% +- 3.92%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 5/10 - acc: 67.1% +- 3.16% - auc: 71.24% +- 5.36%

Iteration: 0
Iteration: 1
Iteration: 2

MISSFOREST RUN 6/10 - acc: 67.67% +- 2.1% - auc: 74.03% +- 4.67%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 7/10 - acc: 69.38% +- 3.35% - auc: 71.46% +- 4.63%

Iteration: 0
Iteration: 1
Iteration: 2

MISSFOREST RUN 8/10 - acc: 68.44% +- 2.48% - auc: 74.01% +- 

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 69.03% +- 2.91% - auc: 74.62% +- 4.66%
SOFTIMPUTE RUN 2/10 - acc: 68.79% +- 3.78% - auc: 72.09% +- 4.06%
SOFTIMPUTE RUN 3/10 - acc: 66.87% +- 3.63% - auc: 70.36% +- 4.01%
SOFTIMPUTE RUN 4/10 - acc: 67.31% +- 2.98% - auc: 69.73% +- 4.65%
SOFTIMPUTE RUN 5/10 - acc: 68.3% +- 2.93% - auc: 71.78% +- 5.5%
SOFTIMPUTE RUN 6/10 - acc: 69.27% +- 2.46% - auc: 73.04% +- 3.48%
SOFTIMPUTE RUN 7/10 - acc: 69.77% +- 2.84% - auc: 75.06% +- 4.36%
SOFTIMPUTE RUN 8/10 - acc: 66.97% +- 2.67% - auc: 72.82% +- 3.79%
SOFTIMPUTE RUN 9/10 - acc: 68.15% +- 1.71% - auc: 71.71% +- 3.99%
SOFTIMPUTE RUN 10/10 - acc: 68.9% +- 3.17% - auc: 73.17% +- 5.39%

SOFTIMPUTE GLOBAL - acc: 68.34% +- 0.95% - auc: 72.44% +- 1.59%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 8.2791

SINKHORN RUN 1/10 - acc: 65.82% +- 2.48% - auc: 71.75% +- 3.07%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 8.3575

SINKHORN RUN 2/10 - acc: 66.02% +- 2.04% - auc: 72.06% +- 2.84%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 8.3516

SINKHORN RUN 3/10 - acc: 67.76% +- 2.83% - auc: 72.35% +- 2.73%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 8.5670

SINKHORN RUN 4/10 - acc: 65.25% +- 1.26% - auc: 70.96% +- 1.84%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 8.3402

SINKHORN RUN 5/10 - acc: 65.04% +- 2.31% - auc: 70.15% +- 4.51%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 8.4171

SINKHORN RUN 6/10 - acc: 66.04% +- 3.67% - auc: 68.03% +- 4.89%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 8.6368

SINKHORN RUN 7/10 - acc: 64.13% +- 1.79% - auc: 67.62% +- 2.01%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 8.5315

SINKHORN RUN 8/10 - acc: 65.54% +- 3.11% - auc: 7

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 68.6% +- 4.3% - auc: 71.87% +- 4.91%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 68.67% +- 2.73% - auc: 73.48% +- 3.26%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 68.87% +- 2.76% - auc: 72.5% +- 4.34%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 67.41% +- 3.49% - auc: 71.57% +- 3.74%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 68.34% +- 3.53% - auc: 71.99% +- 3.46%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 69.74% +- 1.96% - auc: 73.76% +- 3.9%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 66.99% +- 2.98% - auc: 71.49% +- 3.34%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 69.83% +- 4.24% - auc: 73.8% +- 5.74%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc: 6

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly worse than MEDIAN
Metric AUC - OUR METHOD is significantly worse than MEDIAN
Metric ACC - OUR METHOD is significantly worse than MEAN
Metric AUC - OUR METHOD is significantly worse than MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly worse than MICE
Metric AUC - OUR METHOD is significantly worse than MICE
Metric ACC - OUR METHOD is even with MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is even with MIDA
Metric AUC - OUR METHOD is significantly better th